In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import fetch_preprocessed_uniform_data, get_preprocessed_test_data


%pip install tensorflow
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

Note: you may need to restart the kernel to use updated packages.


In [3]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

In [4]:
X_train, y_train, X_val, y_val = fetch_preprocessed_uniform_data()
X_test_estimated_a_processed, X_test_estimated_b_processed, X_test_estimated_c_processed = get_preprocessed_test_data()

c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:357: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample("1H").mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:357: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample("1H").mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:357: FutureWarning: The default value of numeric_o

                    time  pv_measurement       date_forecast  \
4355 2019-12-01 09:00:00            4.40 2019-12-01 09:00:00   
4356 2019-12-01 10:00:00           66.00 2019-12-01 10:00:00   
4357 2019-12-01 11:00:00           41.80 2019-12-01 11:00:00   
4358 2019-12-01 12:00:00            4.62 2019-12-01 12:00:00   
4359 2019-12-01 13:00:00            0.00 2019-12-01 13:00:00   

      absolute_humidity_2m:gm3  air_density_2m:kgm3  clear_sky_energy_1h:J  \
4355                     4.800              1.27200           27331.500000   
4356                     4.825              1.27300           98679.046875   
4357                     4.800              1.27350          142199.703125   
4358                     4.750              1.27525          125991.281250   
4359                     4.650              1.27700           61200.398438   

      clear_sky_rad:W  cloud_base_agl:m  dew_or_rime:idx  dew_point_2m:K  ...  \
4355        17.549999        456.250000              0.0      273

In [5]:
(X_train.shape, X_val.shape, 
 y_train.shape, y_val.shape)

((2783, 51), (696, 51), (2783,), (696,))

In [ ]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i: i + window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [6]:

# Scale the feature data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_Y.fit_transform(y_train.values.reshape(-1, 1))
X_val_scaled = scaler_X.transform(X_train)
y_val_scaled = scaler_Y.transform(y_train.values.reshape(-1, 1))
WINDOW_SIZE = 1
# Reshape input to be [samples, time steps, features]
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], WINDOW_SIZE, X_train_scaled.shape[1]))
X_val_reshaped = X_val_scaled.reshape((X_val_scaled.shape[0], WINDOW_SIZE, X_val_scaled.shape[1]))

# Construct the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val_scaled), verbose=2, shuffle=False)
model.summary()

NameError: name 'X_train_obs_combined' is not defined

In [ ]:

# Make predictions on validation data
y_val_pred_scaled = model.predict(X_val_reshaped)

# Transform predictions back to original scale
y_val_pred = scaler_Y.inverse_transform(y_val_pred_scaled).flatten()
# Calculate Mean Absolute Error on validation data
mae_val = mean_absolute_error(y_val_scaled, y_val_pred)
print(f"MSE on validation data: {mae_val:.2f}")

74/74 [==============================] - 0s 1ms/step
MSE on validation data: 15.05


In [ ]:
# Make predictions using the trained Linear Regression model on the processed test estimated data
from src.features.preprocess_data import get_final_prediction

print(X_test_estimated_a_processed.shape)

print(X_val_reshaped.shape)
scaler_X = MinMaxScaler(feature_range=(0, 1))

X_test_estimated_a_processed = scaler_X.fit_transform(X_test_estimated_a_processed)
X_test_estimated_b_processed = scaler_X.fit_transform(X_test_estimated_b_processed)
X_test_estimated_c_processed = scaler_X.fit_transform(X_test_estimated_c_processed)

X_test_estimated_a_reshaped = X_test_estimated_a_processed.reshape((X_test_estimated_a_processed.shape[0], WINDOW_SIZE, X_test_estimated_a_processed.shape[1]))
X_test_estimated_b_reshaped = X_test_estimated_b_processed.reshape((X_test_estimated_b_processed.shape[0], WINDOW_SIZE, X_test_estimated_b_processed.shape[1]))
X_test_estimated_c_reshaped = X_test_estimated_c_processed.reshape((X_test_estimated_c_processed.shape[0], WINDOW_SIZE, X_test_estimated_c_processed.shape[1]))

y_pred_test_est_a = model.predict(X_test_estimated_a_reshaped)
y_pred_test_est_b = model.predict(X_test_estimated_b_reshaped)
y_pred_test_est_c = model.predict(X_test_estimated_c_reshaped)
# Combine the predictions from the three locations into a single dataframe
y_pred_test_est_a = scaler_Y.inverse_transform(y_pred_test_est_a).flatten()
y_pred_test_est_b = scaler_Y.inverse_transform(y_pred_test_est_b).flatten()
y_pred_test_est_c = scaler_Y.inverse_transform(y_pred_test_est_c).flatten()

y_pred_test_est_combined = get_final_prediction(y_pred_test_est_a, y_pred_test_est_b, y_pred_test_est_c)
y_pred_test_est_combined.head(20)


(2880, 53)
(2343, 1, 53)
90/90 [==============================] - 0s 1ms/step
            date_calc       date_forecast  absolute_humidity_2m:gm3  \
0 2023-04-30 07:00:04 2023-05-01 00:00:00                       4.4   
1 2023-04-30 07:00:04 2023-05-01 00:15:00                       4.3   
2 2023-04-30 07:00:04 2023-05-01 00:30:00                       4.3   
3 2023-04-30 07:00:04 2023-05-01 00:45:00                       4.3   
4 2023-04-30 07:00:04 2023-05-01 01:00:00                       4.3   

   air_density_2m:kgm3  clear_sky_energy_1h:J  clear_sky_rad:W  \
0                1.286                    0.0              0.0   
1                1.287                    0.0              0.0   
2                1.287                    0.0              0.0   
3                1.287                    0.0              0.0   
4                1.287                    0.0              0.0   

   cloud_base_agl:m  dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  ...  \
0       1041.199951   

,id,time,location,prediction
0,0,2023-05-01 00:00:00,A,31.798460
1,1,2023-05-01 01:00:00,A,33.788410
2,2,2023-05-01 02:00:00,A,30.757355
3,3,2023-05-01 03:00:00,A,33.042576
4,4,2023-05-01 04:00:00,A,43.879154
5,5,2023-05-01 05:00:00,A,74.732460
6,6,2023-05-01 06:00:00,A,123.111153
7,7,2023-05-01 07:00:00,A,200.208130
8,8,2023-05-01 08:00:00,A,251.487213
9,9,2023-05-01 09:00:00,A,247.227737


## Step : Save model

In [ ]:

from src.models.saving import save_predictions


save_predictions(y_pred_test_est_combined, 'lstm_only_obs_no_feature_engineering')

   id  prediction
0   0   31.798460
1   1   33.788410
2   2   30.757355
3   3   33.042576
4   4   43.879154
